In [1]:
import subprocess
import os

import pandas as pd

import requests
from bs4 import BeautifulSoup

import json


pd.set_option('display.max_rows', 500)

# 2.1 Data Understanding

The following three options available for extracting data:
- RKI, webscrape [webscrapping] [COVID-19 : Case numbers in Germany and worldwide](https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html)

- John Hopkins, (GITHUB) [COVID-19 Data Repository by John Hopkins](https://github.com/CSSEGISandData/COVID-19.git)

- REST API Services to retrieve data [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/)

## 2.1.1 Jhon Hopkin GITHUB csv data

Clonning data from Johnhopkins GITHUB page
- using command 'git clone/pull' https://github.com/CSSEGISandData/COVID-19.git

In [2]:
# To Automate the Github data pulling
git_pull = subprocess.Popen( "git pull" ,
                         cwd = os.path.dirname( r'C:\Users\SurabhiD\ads_covid_19\data\raw\new-covid' ),
                         #cwd = os.path.dirname( '../Users/SurabhiD/ads_covid_19/data/raw/new-covid' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
(out, error) = git_pull.communicate()


print("Error : " + str(error))
print("out : " + str(out))

Error : b'fatal: not a git repository (or any of the parent directories): .git\n'
out : b''


In [3]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [4]:
pd_raw

Province/State                    Country/Region  \
0                                 NaN                       Afghanistan   
1                                 NaN                           Albania   
2                                 NaN                           Algeria   
3                                 NaN                           Andorra   
4                                 NaN                            Angola   
5                                 NaN               Antigua and Barbuda   
6                                 NaN                         Argentina   
7                                 NaN                           Armenia   
8        Australian Capital Territory                         Australia   
9                     New South Wales                         Australia   
10                 Northern Territory                         Australia   
11                         Queensland                         Australia   
12                    South Australia                         Australia   
13                           Tasmania                         Australia   
14                           Victoria                         Australia   
15                  Western Australia                         Australia   
16                                NaN                           Austria   
17                                NaN                        Azerbaijan   
18                                NaN                           Bahamas   
19                                NaN                           Bahrain   
20                                NaN                        Bangladesh   
21                                NaN                          Barbados   
22                                NaN                           Belarus   
23                                NaN                           Belgium   
24                                NaN                             Benin   
25                                NaN                            Bhutan   
26                                NaN                           Bolivia   
27                                NaN            Bosnia and Herzegovina   
28                                NaN                            Brazil   
29                                NaN                            Brunei   
30                                NaN                          Bulgaria   
31                                NaN                      Burkina Faso   
32                                NaN                        Cabo Verde   
33                                NaN                          Cambodia   
34                                NaN                          Cameroon   
35                            Alberta                            Canada   
36                   British Columbia                            Canada   
37                     Grand Princess                            Canada   
38                           Manitoba                            Canada   
39                      New Brunswick                            Canada   
40          Newfoundland and Labrador                            Canada   
41                        Nova Scotia                            Canada   
42                            Ontario                            Canada   
43               Prince Edward Island                            Canada   
44                             Quebec                            Canada   
45                       Saskatchewan                            Canada   
46                                NaN          Central African Republic   
47                                NaN                              Chad   
48                                NaN                             Chile   
49                              Anhui                             China   
50                            Beijing                             China   
51                          Chongqing                             China   
52                             Fujian           

## 2.1.1 Webscrapping

- RKI, webscrape (webscraping) [Robert-koch website fälle](https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html)

In [5]:
page = requests.get("https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Fallzahlen.html")

In [6]:
soup = BeautifulSoup(page.content, 'html.parser')

In [7]:
#soup.get_text()

In [8]:
html_table=soup.find('table')

In [9]:
all_rows=html_table.find_all('tr')

In [10]:
final_data_list=[]

In [11]:
for pos,rows in enumerate(all_rows):
#print(pos)
    col_list=[each_col.get_text(strip=True) for each_col in rows.find_all('td')]
    final_data_list.append(col_list)
#print(rows)
#print(col_list)

In [12]:
pd_daily_status = pd.DataFrame(final_data_list).dropna().rename(columns={0:'State',
                                                    1:'Cases',
                                                    2:'Changes',
                                                    3:'Cases_per_100K',
                                                    4: 'Fatal',
                                                    5:'Comment'})

In [13]:
pd_daily_status.head()

State   Cases Changes Cases_per_100K Fatal Comment
2  Baden-Württem­berg  40.358    +185          1.450  13,1   1.863
3              Bayern  55.414    +353          1.835  14,0   2.634
4              Berlin  10.786     +58            416  11,1     226
5        Branden­burg   3.795      +8             70   2,8     169
6              Bremen   1.913      +8             62   9,1      56

In [14]:
pd_daily_status.to_csv('../data/raw/RKI/RKI_data.csv',sep=';', index = False) 
# Data will be prepared in notebook 'data_preparation'

# 2.3.1 REST API Calls 

REST API Services to retrieve data [NPGEO Corona Hub 2020](https://npgeo-corona-npgeo-de.hub.arcgis.com/)

In [15]:
#Data Request for Country: Germany
data=requests.get('https://services7.arcgis.com/mOBPykOjAyBO2ZKk/arcgis/rest/services/Coronaf%C3%A4lle_in_den_Bundesl%C3%A4ndern/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json')

In [16]:
#data.content

In [17]:
json_object=json.loads(data.content)

In [18]:
type(json_object)

dict

In [19]:
json_object.keys()

dict_keys(['objectIdFieldName', 'uniqueIdField', 'globalIdFieldName', 'geometryProperties', 'serverGens', 'geometryType', 'spatialReference', 'fields', 'features'])

In [20]:
#json_object

In [21]:
full_list=[]
for pos,each_dict in enumerate (json_object['features'][:]):
    full_list.append(each_dict['attributes'])

In [22]:
pd_full_list=pd.DataFrame(full_list)
pd_full_list.head()

OBJECTID_1 LAN_ew_AGS           LAN_ew_GEN            LAN_ew_BEZ  \
0           1         01   Schleswig-Holstein                  Land   
1           2         02              Hamburg  Freie und Hansestadt   
2           3         03        Niedersachsen                  Land   
3           4         04               Bremen      Freie Hansestadt   
4           5         05  Nordrhein-Westfalen                  Land   

   LAN_ew_EWZ  OBJECTID  Fallzahl  Aktualisierung AGS_TXT  \
0     2896712        15      3927   1598306400000      01   
1     1841179         6      6083   1598306400000      02   
2     7982448         9     16191   1598306400000      03   
3      682986         5      1913   1598306400000      04   
4    17932651        10     57213   1598306400000      05   

                               GlobalID  faelle_100000_EW   Shape__Area  \
0  fc5ba936-c95c-432c-8a33-9eb2f30b660f        135.567499  4.573731e+10   
1  0f3e860c-5181-4d3f-a421-1d51f50315ea        330.386128  2.089396e+09   
2  3fd77024-c29b-4843-9be8-682ad48e60c9        202.832515  1.299836e+11   
3  4132268b-54de-4327-ac1e-760e915112f1        280.093589  1.119157e+09   
4  561d658f-3ee5-46e3-bc95-3528c6558ab9        319.043738  8.782936e+10   

   Shape__Length  Death  
0   2.881496e+06    160  
1   4.188002e+05    265  
2   4.008988e+06    661  
3   3.357177e+05     56  
4   2.648673e+06   1802

In [23]:
pd_full_list.to_csv('../data/raw/NPGEO/GER_state_data.csv',sep=';')

In [24]:
pd_full_list.shape[0]

16

# 2.4.1 API access via REST service, e.g. India Data

example of REST conform interface (Registration Required)  
[Smartable.ai](https://smartable.ai/)

In [25]:
url_endpoint='https://api.smartable.ai/coronavirus/stats/IN'
headers={
    'Cache-Control':'no-cache',
    'Subscription-Key': '096aaad7799c49b3825e698f67e67e2c',
}
response = requests.get(url_endpoint,headers=headers)

In [26]:
print(response)

<Response [200]>


In [27]:
IN_dict = json.loads(response.content) # imports string
with open('../data/raw/smartable/IN_data.txt', 'w') as outfile:
    json.dump(IN_dict, outfile, indent=2)

In [28]:
print(json.dumps(IN_dict,indent=2)) #string dump

{
  "location": {
    "long": 78.0,
    "countryOrRegion": "India",
    "provinceOrState": null,
    "county": null,
    "isoCode": "IN",
    "lat": 21.0
  },
  "updatedDateTime": "2020-08-25T22:00:34.6092863Z",
  "stats": {
    "totalConfirmedCases": 3231754,
    "newlyConfirmedCases": 66873,
    "totalDeaths": 59612,
    "newDeaths": 1066,
    "totalRecoveredCases": 2467252,
    "newlyRecoveredCases": 64151,
    "history": [
      {
        "date": "2020-01-22T00:00:00",
        "confirmed": 0,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-23T00:00:00",
        "confirmed": 0,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-24T00:00:00",
        "confirmed": 0,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-25T00:00:00",
        "confirmed": 0,
        "deaths": 0,
        "recovered": 0
      },
      {
        "date": "2020-01-26T00:00:00",
        "confirmed": 0,
  

### 2.4.1.1 Individual States India

In [29]:
IN_dict['stats']['breakdowns'][0]

{'location': {'long': 78.0,
  'countryOrRegion': 'India',
  'provinceOrState': None,
  'county': None,
  'isoCode': 'IN',
  'lat': 21.0},
 'totalConfirmedCases': 3231754,
 'newlyConfirmedCases': 66873,
 'totalDeaths': 59612,
 'newDeaths': 1066,
 'totalRecoveredCases': 2467252,
 'newlyRecoveredCases': 64151}

In [30]:
full_list_IN_country=[]
for pos,each_dict in enumerate (IN_dict['stats']['breakdowns'][:]):
    flatten_dict=each_dict['location']
    flatten_dict.update(dict(list(IN_dict['stats']['breakdowns'][pos].items())[1: 7]) 
    )
    full_list_IN_country.append(flatten_dict)

In [31]:
pd.DataFrame(full_list_IN_country).to_csv('../data/raw/smartable/full_list_IN_country.csv',sep=';',index=False)